In [1]:
!pip install transformers datasets peft torch pandas


In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

# Load the dataset from CSV
data = pd.read_csv('/content/drive/MyDrive/Summarization-dataset/tourism_articles.csv')

# Ensure the dataset has 'article' and 'summary' columns
assert 'article' in data.columns and 'summary' in data.columns

# Drop the 'id' and 'title' columns if they exist
columns_to_drop = ['id', 'title']
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns])

# Split the dataset into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert the pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Combine them into a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

# Load the tokenizer
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocessing function
def preprocess_function(examples):
    inputs = examples['article']
    targets = examples['summary']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(targets, max_length=84, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the preprocessing function to the datasets
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Remove the original columns to avoid confusion
tokenized_datasets = tokenized_datasets.remove_columns(['article', 'summary'])

# Set the format for PyTorch
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Save the tokenized dataset if needed
tokenized_datasets.save_to_disk('./tokenized_dataset')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pyth

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/128 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/32 [00:00<?, ? examples/s]

In [3]:
print(dataset['train'][0])
print(dataset['test'][0])

{'article': 'ساختمان موراکا در سنگاپور ساخته\u200c شده است و توسط مهندسین نیوزلندی و کارشناسان فن\u200cآوری آکواریوم شرکت ام جی مورفی طراحی\u200c شده است و درنهایت به آب\u200cهای گرم مالدیو انتقال پیدا کرده است. در این سازه ۶ تنی و دوطبقه، طبقه فوقانی به اتاق نشیمن، آشپزخانه، ناهارخوری، باشگاه و استخر اختصاص دارد و از طریق یک \u200cراه\u200cپله مارپیچی به اتاق\u200cخواب طبقه زیرین دسترسی پیدا می\u200cکند. قسمت زیر دریای این اقامتگاه نیز (یکی از اتاق\u200cخواب\u200cها) در عمق ۵ متری زیر آب قرارگرفته است. \n شرکت طراحی یوجی یامازاکی که در نیویورک آمریکا مستقر است طراحی داخلی موراکا را بر عهده داشته است و تئوری تیم آن\u200cها هنگام طراحی به این صورت بوده است که بیان می\u200cکردند: مردم برای دیدن تئاتر بهک تئاتر نمی\u200cروند، بلکه برای دیدن نمایش به تئاتر می\u200cروند. \n در طراحی مدرن طبقه فوقانی پس\u200cزمینه سفیدرنگ را با استفاده از مواد طبیعی و مبلمانی ساده به کار گرفته\u200cاند. پنجره\u200cهای این بخش به\u200cصورت تمام\u200cقد است که چشم\u200cانداز کاملی از اقیانوس را در اختیار مهمان

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset, load_metric
import torch

# Load the model and tokenizer
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


# Define the LoRA configuration
lora_config = LoraConfig(
    r=4,
    lora_alpha=32,
    task_type=TaskType.SEQ_2_SEQ_LM,
    lora_dropout=0.1,
    bias='none',
)

# Apply the LoRA configuration to the model
model = get_peft_model(model, lora_config)


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.662491
2,No log,2.661693
3,No log,2.661358


TrainOutput(global_step=96, training_loss=2.5153770446777344, metrics={'train_runtime': 3080.7219, 'train_samples_per_second': 0.125, 'train_steps_per_second': 0.031, 'total_flos': 460955553103872.0, 'train_loss': 2.5153770446777344, 'epoch': 3.0})

In [8]:
model.save_pretrained("/content/drive/MyDrive/fine-tuning-project/task1/fine-tuned-task1")
tokenizer.save_pretrained("/content/drive/MyDrive/fine-tuning-project/task1/fine-tuned-task1")


/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:399: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/MyDrive/fine-tuning-project/task1/fine-tuned-task1/tokenizer_config.json',
 '/content/drive/MyDrive/fine-tuning-project/task1/fine-tuned-task1/special_tokens_map.json',
 '/content/drive/MyDrive/fine-tuning-project/task1/fine-tuned-task1/spiece.model',
 '/content/drive/MyDrive/fine-tuning-project/task1/fine-tuned-task1/added_tokens.json',
 '/content/drive/MyDrive/fine-tuning-project/task1/fine-tuned-task1/tokenizer.json')

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the fine-tuned model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/fine-tuning-project/task1/fine-tuned-task1")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/fine-tuning-project/task1/fine-tuned-task1")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
# Define a sample input
sample_article = """به گزارش روابط عمومی صباایده، این برند فعال در حوزه ویدئو، استریم و تبلیغات آنلاین که پلتفرمهایی چون فیلیمو، آپارات و آژانس تبلیغات آنلاین صباویژن را زیر مجموعه خود دارد، با سرمایه گذاری در یکی از دو استارتاپ بزرگ حوزه گردشگری، به طور رسمی وارد بازار فروش و رزرواسیون بلیط هواپیما و قطار شد.
 ظرفیت‌های موجود در بازار گردشگری آنلاین ایران و فاصله زیادی که این حوزه از بازار آفلاین دارد، رشد قابل‌توجه و ارگانیک فلایتیو از ابتدای فعالیت آن و ویژگی‌های مشترک تیم‌های مدیریتی دو برند که آن‌ها را بهم نزدیک‌تر می‌کند از مهمترین علت‌های جدیدترین سرمایه‌گذاری صباایده است.
 این سرمایه‌گذاری که با خرید ۲۰ درصد سهام فلایتیو رقم خورده پیرو تجربه و دانش فلایتیو در صنعت گردشگری و در کنار ظرفیت‌ها و حمایت‌های گسترده تبلیغاتی صباایده، مسیر تازه‌ای را در صنعت گردشگری آنلاین ایران رقم خواهد زد.
"""

# Preprocess the input
input_ids = tokenizer(
    [sample_article],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

# Generate the summary
output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

# Decode the summary
summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

صباایده، یکی از بزرگترین شرکت های گردشگری آنلاین ایران، با سرمایه گذاری بیش از ۲۰ درصد سهام خود در بازار فروش و رزرواسیون بلیط هواپیما و قطار وارد بازار شد.
